In [30]:
import numpy as np
import pandas as pd
pd.options.plotting.backend = "plotly"
from IPython.display import display
import plotly.express as px

In [31]:
fname = "Logdata Essay mini60 2023-10-31.csv"

def load_krp_file(fname):
    nrows = 12
    df_head = pd.read_csv(fname, nrows=nrows, names=["key", "value"])
    df_head = df_head.set_index("key")
    df = pd.read_csv(fname, skiprows=nrows)
    units = df.iloc[0, :]
    units.name = "units"
    cols = df.columns
    df = df.drop(0)
    for col in df.columns:
        df[col] = df[col].astype(float)
    df["Lap"] = ((df["Distance"] - df["Distance"].shift()) < 0).astype(int).cumsum()
    #df["Laptime"] = pd.NaT
    df["Starttime"] = np.where((df["Distance"] - df["Distance"].shift()) < 0, df["Time"], np.NaN)
    df["Starttime"].iloc[0] = 0
    df["Starttime"] = df["Starttime"].fillna(method="ffill")
    df["Laptime"] = df["Time"] - df["Starttime"]

    df["Time"] = pd.to_datetime(df["Time"], unit="s")
    #df["Time"] = pd.to_timedelta(df["Time"], unit="s")

    #df["Laptime"] = pd.to_datetime(df["Laptime"], unit="s")
    
    df = df.set_index("Time")
    laps = df["Lap"].unique()
    
    laptimes = df.groupby("Lap")["Laptime"].last()[0:-1]
    
    return df_head, units.to_frame(), df, laps, laptimes

def get_lap_data(df, lap):
    df_lap = df[df["Lap"] == lap]
    df_lap = df_lap.set_index("Laptime")
    return df_lap

def plot_lap_data(df, values="Engine"):
    df_lap = get_lap_data(df, 0)
    fig = px.line(df_lap, x=df_lap.index, y=values, color='Lap')
    for lap in laps[1:]:
        df_lap = get_lap_data(df, lap)
        fig.add_scatter(x=df_lap.index, y=df_lap[values], mode='lines', name=str(lap))
    fig.show()

df_head, df_units, df, laps, laptimes = load_krp_file(fname)
display(df_head)
display(df_units)
display(df)
display(laptimes.sort_values().to_frame())

C:\Users\scell\AppData\Local\Temp\ipykernel_13544\2153780786.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\scell\AppData\Local\Temp\ipykernel_13544\2153780786.py:18: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



,value
key,
Format,PiBoSo CSV File
Venue,Circuit International d'Essay
Vehicle,CRG Mini60
User,Antoine Celles
Data Source,Kart Racing Pro
Comment,NaN
Date,10/31/23
Time,18:37:43
Sample Rate,10


,units
Time,s
Distance,m
Engine,rpm
CylHeadTemp,C
WaterTemp,C
Gear,NaN
Speed,km/h
LatAcc,G
LonAcc,G
Steer,deg


,Distance,Engine,CylHeadTemp,WaterTemp,Gear,Speed,LatAcc,LonAcc,Steer,Throttle,Brake,FrontBrakes,Clutch,YawVel,PosX,PosY,Lap,Starttime,Laptime
Time,,,,,,,,,,,,,,,,,,,
1970-01-01 00:00:00.002,43.049,2198.0,124.991264,0.0,0.0,0.070633,0.002549,0.000202,-1.173652,16.666660,0.00000,0.0,0.0,0.000484,35.729687,136.618561,0,0.000,0.002
1970-01-01 00:00:00.101,43.049,2177.0,124.556961,0.0,0.0,0.545799,0.001613,0.014398,-1.173652,18.562508,0.00000,0.0,0.0,0.059395,35.729839,136.618393,0,0.000,0.101
1970-01-01 00:00:00.200,43.049,2176.0,124.134369,0.0,0.0,0.501256,-0.008634,0.023318,-1.173652,18.588047,0.00000,0.0,0.0,-0.042611,35.729771,136.617615,0,0.000,0.200
1970-01-01 00:00:00.300,43.049,2176.0,123.705948,0.0,0.0,0.304337,-0.000904,0.016425,-1.173652,18.588333,0.00000,0.0,0.0,-0.009988,35.729748,136.616974,0,0.000,0.300
1970-01-01 00:00:00.400,43.049,2176.0,123.280342,0.0,0.0,0.155313,0.007863,0.012276,-1.173652,18.588333,0.00000,0.0,0.0,-0.037215,35.729797,136.615738,0,0.000,0.400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1970-01-01 00:04:56.900,82.974,8986.0,103.760742,0.0,0.0,65.771088,1.100487,-0.324477,12.073339,0.000000,1.17302,0.0,0.0,28.847034,71.743393,98.704651,5,293.401,3.499
1970-01-01 00:04:57.000,85.098,8750.0,103.411682,0.0,0.0,64.806839,0.658116,-0.194999,10.702374,0.000000,1.17302,0.0,0.0,23.109797,72.994591,97.395935,5,293.401,3.599
1970-01-01 00:04:57.101,87.192,8632.0,103.064720,0.0,0.0,63.726986,1.066653,-0.374704,12.521113,0.000000,1.17302,0.0,0.0,36.464943,74.173767,96.055145,5,293.401,3.700


,Laptime
Lap,
2,56.600
4,56.799
3,57.401
1,57.699
0,64.401


In [32]:
df["Distance"].plot()

C:\Users\scell\AppData\Roaming\Python\Python310\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [33]:
df["Engine"].plot()

C:\Users\scell\AppData\Roaming\Python\Python310\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [34]:
plot_lap_data(df, "Engine")

In [35]:
data_cols = df.columns.drop(["Distance", "Lap", "Laptime", "Starttime", "PosX", "PosY"])
data_cols

Index(['Engine', 'CylHeadTemp', 'WaterTemp', 'Gear', 'Speed', 'LatAcc',
       'LonAcc', 'Steer', 'Throttle', 'Brake', 'FrontBrakes', 'Clutch',
       'YawVel'],
      dtype='object')

In [36]:
from IPython.display import display, HTML
display(HTML("<style>div.output_scroll { height: 44em; }</style>"))

In [37]:
for data_col in data_cols:
    plot_lap_data(df, data_col)

In [38]:
fig = px.scatter(df, x="PosX", y="PosY", symbol="Lap", hover_name="Laptime")
fig.show()